In [1]:
import pandas as pd
import os

# Get all file paths in the Nutritional_value_Data folder
paths = [
    os.path.join("raw_data", filename)
    for filename in os.listdir("raw_data")
    if os.path.isfile(os.path.join("raw_data", filename))
]

print(paths)

['raw_data/nutrient_value_some_common_foods_soups-2008-valeur_nutritive_quelques_aliments_usuels_soupe-eng.csv', 'raw_data/nutrient_value_miscellaneous_foods_2008_valeur_nutritive_divers-eng.csv', 'raw_data/nutrient_value_dairy_foods_and_other_related_products_2008_valeur_nutritive_produits_laitiers_et_produits_annexes-eng.csv', 'raw_data/nutrient_value_legumes_nuts_and_seeds_2008_valeur_nutritive_legumineuses_noix_et_graines-eng.csv', 'raw_data/nutrient_value_meat_and_poultry_2008_valeur_nutritive_2008_viande_et_volaille-eng.csv', 'raw_data/nutrient_value_vegetables_and_vegetable_products_2008_valeur_nutritive_legumes_et_produits_base_de_legumes-eng.csv', 'raw_data/nutrient_value_some_common_foods_sweets_sugars-2008-valeur_nutritive_quelques_aliments_usuels_sucres_sucreries-eng.csv', 'raw_data/nutrient_value_mixed_dishes_2008_valeur_nutritive_mets_composes-eng.csv', 'raw_data/nutrient_value_fats_and_oils_foods_2008_valeur_nutritive_matieres_grasses_et_huiles-eng.csv', 'raw_data/nutrie

In [2]:
# Leave only columns that are present in all the files
# The data is not very clean, and usually there are two cases how to clearn the data
common_cols = None

for path in paths:
    df = pd.read_csv(path, on_bad_lines='skip', encoding='ISO-8859-1')
    df = df.dropna(how='all')
    if df.iloc[1, 0] == 'Food name':
        df = df.iloc[1:, :]
        df.columns = df.iloc[0]
        df = df[1:].reset_index(drop=True)
        category = df.iloc[0, 0]
        df = df.iloc[1:, :]
        df['Food Category'] = category
        df = df.loc[:, df.columns.notna()]
        cols = set(df.columns)
        if common_cols is None:
            common_cols = cols
        else:
            common_cols = common_cols.intersection(cols)
    elif df.iloc[2, 0] == 'Food name':
        df = df.iloc[1:, :]
        category = df.iloc[0, 0]
        df = df.iloc[1:, :]
        df.columns = df.iloc[0]
        df = df[3:].reset_index(drop=True)
        df['Food Category'] = category
        df = df.loc[:, df.columns.notna()]
        cols = set(df.columns)
        if common_cols is None:
            common_cols = cols
        else:
            common_cols = common_cols.intersection(cols)
common_cols = list(common_cols)
common_cols.insert(0, common_cols.pop(common_cols.index('Food Category')))
common_cols.insert(1, common_cols.pop(common_cols.index('Food name')))
common_cols.insert(2, common_cols.pop(common_cols.index('Weight ')))
common_cols.insert(3, common_cols.pop(common_cols.index('Measure')))
common_cols

['Food Category',
 'Food name',
 'Weight ',
 'Measure',
 'Energy',
 'Phosphorus',
 'Sodium',
 'Total Fat',
 'Carbohydrate',
 'Potassium',
 'Iron']

In [3]:
# present not everywhere but we need it
common_cols.append('Protein')
common_cols.insert(4, common_cols.pop(common_cols.index('Protein')))
common_cols.insert(5, common_cols.pop(common_cols.index('Carbohydrate')))
common_cols.insert(6, common_cols.pop(common_cols.index('Total Fat')))
common_cols

['Food Category',
 'Food name',
 'Weight ',
 'Measure',
 'Protein',
 'Carbohydrate',
 'Total Fat',
 'Energy',
 'Phosphorus',
 'Sodium',
 'Potassium',
 'Iron']

In [4]:
# Combine all the individual files into one big dataframe
# The data is not very clean, and usually there are two cases how to clearn the data
big_df = pd.DataFrame()
problematic_paths = []
for path in paths:
    try:
        df = pd.read_csv(path, on_bad_lines='skip', encoding='ISO-8859-1')
        df = df.dropna(how='all')
        if df.iloc[1, 0] == 'Food name':
            print('good (case 1)')
            df = df.iloc[1:, :]
            df.columns = df.iloc[0]
            df = df[1:].reset_index(drop=True)
            category = df.iloc[0, 0]
            df = df.iloc[1:, :]
            df['Food Category'] = category
            df = df.loc[:, df.columns.notna()]
            cols = df.columns.tolist()
            if 'Protein' not in cols:
                print('No protein!')
                cols.append('Protein')
                df['Protein'] = ''
            cols.insert(0, cols.pop(cols.index('Food Category')))
            df = df[cols]
            df = df[common_cols]
            big_df = pd.concat([big_df, df])
        elif df.iloc[2, 0] == 'Food name':
            print('good (case 2)')
            df = df.iloc[1:, :]
            category = df.iloc[0, 0]
            df = df.iloc[1:, :]
            df.columns = df.iloc[0]
            df = df[3:].reset_index(drop=True)
            df['Food Category'] = category
            df = df.loc[:, df.columns.notna()]
            cols = df.columns.tolist()
            if 'Protein' not in cols:
                print('No protein!')
                cols.append('Protein')
                df['Protein'] = ''
            cols.insert(0, cols.pop(cols.index('Food Category')))
            df = df[cols]
            df = df[common_cols]
            big_df = pd.concat([big_df, df])
        else:
            print('error')
            problematic_paths.append(path)
    except Exception as e:
        print(str(e))
        problematic_paths.append(path)

good (case 2)
good (case 2)
good (case 1)
good (case 2)
good (case 2)
good (case 1)
good (case 2)
good (case 2)
No protein!
good (case 2)
good (case 1)
good (case 1)
good (case 2)
good (case 1)
good (case 2)
good (case 1)
good (case 2)
good (case 2)


In [5]:
big_df

,Food Category,Food name,Weight,Measure,Protein,Carbohydrate,Total Fat,Energy,Energy,Energy,Energy,Phosphorus,Sodium,Potassium,Iron
0,SOUPS,"Beef or chicken, broth/bouillon",254,250mL,3,tr,1,18,74,18,74,33,827,137,0.4
1,SOUPS,"Beef, chunky",254,250mL,12,21,5,180,753,180,753,127,915,355,2.5
2,SOUPS,"Chicken noodle, chunky",254,250mL,13,18,6,185,773,185,773,76,898,114,1.5
3,SOUPS,"Chicken noodle, low fat, reduced salt",257,250mL,9,13,2,105,444,105,444,NaN,501,NaN,0
4,SOUPS,"Chicken vegetable, chunky",254,250mL,13,20,5,175,733,175,733,112,1128,388,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,SNACKS,"Rice cakes, plain",9,1,1,7,tr,35,146,35,146,32,29,26,0.1
21,SNACKS,"Sesame sticks, salted",14,60mL,2,6,5,75,316,75,316,19,208,25,0.1
22,SNACKS,"Soybeans, roasted, salted",44,60mL,15,15,11,205,859,205,859,158,71,641,1.7
23,SNACKS,"Trail mix, regular",38,60mL,5,17,11,176,735,176,735,131,87,261,1.2


In [6]:
big_df.to_csv('Food_all.csv', index=False)